In [23]:
import torch
import platform
platform = platform.system()
if platform == 'Linux':
    !BASICSR_JIT=True
    pretrained_discriminator = "../experiments/ridcp_first_train/models/net_d_best_.pth"
    pretrained_hqp_path = "../pretrained_models/pretrained_HQPs_renamed.pth"
    pretrained_net_path = "../pretrained_models/pretrained_RIDCP_renamed.pth"
else:
    !set BASICSR_JIT=True
    pretrained_discriminator = "../experiments/ridcp_first_train/models/net_d_best_.pth"
    pretrained_hqp_path = "../pretrained_models/pretrained_HQPs_renamed.pth"
    pretrained_net_path = "../pretrained_models/pretrained_RIDCP_renamed.pth"

In [24]:
from basicsr import GANLoss
from PIL import Image
from torchvision.transforms import ToTensor
from basicsr.archs.ridcp_new_arch import RIDCPNew
from basicsr.archs.ridcp.discriminator import UNetDiscriminatorSN

In [25]:
discriminator = UNetDiscriminatorSN(512)
discriminator.cuda()
discriminator.load_state_dict(torch.load(pretrained_discriminator)["params"])
discriminator.eval()

hqp = RIDCPNew(LQ_stage=False)
hqp.cuda()
hqp.load_state_dict(torch.load(pretrained_hqp_path)["params"])
hqp.eval()

ridcp = RIDCPNew(LQ_stage=True)
ridcp.cuda()
ridcp.load_state_dict(torch.load(pretrained_net_path)["params"])
ridcp.eval()

RIDCPNew(
  (vq_encoder): VQEncoder(
    (in_conv): Conv2d(3, 64, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1))
    (blocks): ModuleList(
      (0): Sequential(
        (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (1): ResBlock(
          (conv): Sequential(
            (0): NormLayer(
              (norm): GroupNorm(32, 128, eps=1e-06, affine=True)
            )
            (1): ActLayer(
              (func): SiLU(inplace=True)
            )
            (2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (3): NormLayer(
              (norm): GroupNorm(32, 128, eps=1e-06, affine=True)
            )
            (4): ActLayer(
              (func): SiLU(inplace=True)
            )
            (5): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          )
        )
        (2): ResBlock(
          (conv): Sequential(
            (0): NormLayer(
              (norm): GroupNorm(32, 128, eps=1e-06

In [26]:
inputs = ToTensor()(Image.open('../datasets/rgb_500/0001.jpg').convert('RGB')).cuda()[None, ::]
inputs.shape


torch.Size([1, 3, 413, 550])

In [27]:
inputs = torch.randn(1, 3, 512, 512).cuda()
out_img, out_img_residual, codebook_loss, feat_to_quant, z_quant, indices = ridcp(inputs)
_, _, _, hqp_feat, _, _ = hqp(inputs)

NameError: name 'deform_conv_ext' is not defined

In [ ]:
out = discriminator(feat_to_quant)
hqp_out = discriminator(hqp_feat)

print(out.mean().item())
print(hqp_out.mean().item())

gan_loss = GANLoss('hinge')

print(gan_loss(out, False, True))
print(gan_loss(hqp_out, True, True))